# Advent of Code 2018

## Day 8: Memory Maneuver


https://adventofcode.com/2018/day/8

The tree is made up of nodes; a single, outermost node forms the tree's root, and it contains all other nodes in the tree (or contains nodes that contain nodes, and so on).

Specifically, a node consists of:

* A header, which is always exactly two numbers:
    - The quantity of child nodes.
    - The quantity of metadata entries.
* Zero or more child nodes (as specified in the header).
* One or more metadata entries (as specified in the header).

Each child node is itself a node that has its own header, child nodes, and metadata. For example:

```
2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2
A----------------------------------
    B----------- C-----------
                     D-----```


In [10]:
(require racket)

We're faced with a tree made up of numbers, expressed as a depth-first traversal. Seems like good Racket terrain. First step is to load up the data.

In [2]:
(define data (map string->number (string-split (first (file->lines "data/input2018-08.data")))))

In [3]:
(take data 10)

'(9 11 6 3 5 3 3 7 1 6)

Looks like we get what we wanted -- a list of numbers.

The tree is a node, which has a (potentially empty) set of child-nodes, and a "metadata" weight. We can represent this as a Racket `struct`

In [4]:
(struct node (children meta) #:transparent)

We need to consume the input list of numbers one at a time -- let's use a generator for this

In [6]:
(require racket/generator)

(define next-item
  (generator () (for ([x (in-list data)]) (yield x))))

Now we can drill down the data given, recursively constructing nodes.

In [7]:
(define (read-node)
  (cond [(equal? (generator-state next-item) 'done) '()]
        [else
         (define-values (child-count meta-count) (values (next-item) (next-item)))
         (node 
          (for/list ([n (in-range child-count)]) (read-node))
          (for/list ([n (in-range meta-count)]) (next-item)))]))

In [8]:
(define tree (read-node))

## Part1: What is the sum of all metadata entries?

For this we traverse the tree and sum up the meta part of the nodes. We could have done that directly in the `read-node` function if we wanted, but let's keep it separate for clarity.

In [37]:
(require math) ; sum

(define (metadata-sum tree)
  ;; The meta data sum is the sum of the current node's meta data list
  ;; plus the sum of its childrens' meta data sums.
  (cond [(null? tree) 0]
        [else
         (+ (sum (node-meta tree))
            (for/sum ([ch (in-list (node-children tree))])
              (metadata-sum ch)))]))

In [12]:
(require rackunit)

In [13]:
(check-eq? (metadata-sum tree) 44838)

Bingo.

## Part2: What's the root node value?

The value of a node depends on whether it has child nodes.

If a node has no child nodes, its value is the sum of its metadata entries.

However, if a node does have child nodes, the metadata entries become indexes which refer to those child nodes. A metadata entry of 1 refers to the first child node, 2 to the second, 3 to the third, and so on. The value of this node is the sum of the values of the child nodes referenced by the metadata entries. If a referenced child node does not exist, that reference is skipped. A child node can be referenced multiple time and counts each time it is referenced. A metadata entry of 0 does not refer to any child node.

We'll need two functions. Firstly, we need a way to get the node value, and then we need a function to pick out a child node's value by its index.

In [35]:
(define (node-value tree)
  (cond [(null? tree) 0]
        [(null? (node-children tree)) (sum (node-meta tree))] ; node has no children: sum-of-meta
        [else
         (for/sum ([i (in-list (node-meta tree))])            ; sum of indexed child nodes' values
           (value@index tree i))]))

(define (value@index cur idx)
  (cond [(zero? idx) 0]
        [(>= (sub1 idx) (length (node-children cur))) 0]
        [else 
         (node-value (list-ref (node-children cur) (sub1 idx)))]))


In [36]:
(check-eq? (part2 tree) 22198)

All done.